In [ ]:
# From SAV to LOVE

Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

2) Изменим регистр твитов на нижний с помощью .lower().

3) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

4) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

5) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

7) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

8) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

9) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

10) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

11) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

12) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

13) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

14) Сохраним результат предобработки в pickle-файл.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [29]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

pd.set_option('max_colwidth', 900)

In [2]:
train_df = pd.read_csv('../../data/nlp/train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
test_df = pd.read_csv('../../data/nlp/test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [4]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [5]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


In [6]:
def user_remove(df):
    return df.replace(regex=r'@[\w]*', value=' ', inplace=True)

user_remove(combine_df)

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


3. Изменим регистр твитов на нижний с помощью .lower()

In [7]:
combine_df['tweet'].str.lower()

0           when a father is dysfunctional and is so se...
1            thanks for #lyft credit i can't use cause ...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
                               ...                        
49154    thought factory: left-right polarisation! #tru...
49155    feeling like a mermaid ð #hairflip #neverre...
49156    #hillary #campaigned today in #ohio((omg)) &am...
49157    happy, at work conference: right mindset leads...
49158    my   song "so glad" free download!  #shoegaze ...
Name: tweet, Length: 49159, dtype: object

4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [8]:
apostrophe_dict = { "aren't": "are not",
                    "can't": "cannot",
                    "couldn't": "could not",
                    "didn't": "did not",
                    "doesn't": "does not",
                    "don't": "do not",
                    "hadn't": "had not",
                    "hasn't": "has not",
                    "haven't": "have not",
                    "he'd": "he had",
                    "he'll": "he will",
                    "he's": "he is",
                    "I'd": "I would",
                    "I'll": "I will",
                    "I'm": "I am",
                    "I've": "I have",
                    "isn't": "is not",
                    "let's": "let us",
                    "she'd": "she had",
                    "she'll": "she will",
                    "she's": "she is",
                    "shouldn't": "should not",
                    "there's": "there is",
                    "they'd": "they had",
                    "they'll": "they will",
                    "they're": "they are",
                    "they've": "they have",
                    "we'd": "we had",
                    "we're": "we are",
                    "we've": "we have", }

In [9]:
def apostrophe_replace(df):
    for index, row in df.iterrows():
        text = ''
        for word in row['tweet'].split():
            for key, value in apostrophe_dict.items():
                if word == key:
                    word = value
            text = ' '.join((text, word))
        df.iloc[index, 2] = text
            
apostrophe_replace(combine_df)

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so self...
1,2,0.0,thanks for #lyft credit i cannot use cause th...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in u...
4,5,0.0,factsguide: society now #motivation


6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

Твит с эмотиконом

In [19]:
combine_df[127:130]

,id,label,tweet
127,128,0.0,sad in the branches itâs just rainy day writing tears are flying birds #haiku #3lines #micropoetry
128,129,0.0,yeah! new buttons in the mail for me ð they are so pretty! :) #jewelrymaking #buttons
129,130,0.0,driver hit female moose on river rd #weston. moose was killed. driver is ok. crews removing animal now


In [12]:
emoticons = [':)', ':-)', ':=)', ';)', ';-)', ';=)', ':D', ':-D', ':-d', ':(', ':-(']

In [35]:
def emoticons_replace(df):
    for index, row in df.iterrows():
        text = ''
        for word in row['tweet'].split():
            for emoticon in emoticons:
                if word == emoticon:
                    word = ' '
            text = ' '.join((text, word))
        df.iloc[index, 2] = text
    return df
emoticons_replace(combine_df)

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0.0,thanks for lyft credit i cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0.0,bihday your majesty
3,4,0.0,model i love u take with u all the time in ur
4,5,0.0,factsguide society now motivation


Проверяем твит с эмтоиконом

In [21]:
combine_df[127:130]

,id,label,tweet
127,128,0.0,sad in the branches itâs just rainy day writing tears are flying birds #haiku #3lines #micropoetry
128,129,0.0,yeah! new buttons in the mail for me ð they are so pretty! #jewelrymaking #buttons
129,130,0.0,driver hit female moose on river rd #weston. moose was killed. driver is ok. crews removing animal now


7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [25]:
combine_df[8:9]

,id,label,tweet
8,9,0.0,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦


In [27]:
def punctuation_replace(df):
    return df.replace(regex=r'[^\w\s]', value=' ', inplace=True)

punctuation_replace(combine_df)

combine_df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0.0,thanks for lyft credit i cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0.0,bihday your majesty
3,4,0.0,model i love u take with u all the time in urð ð ð ð ð ð ð ð
4,5,0.0,factsguide society now motivation
5,6,0.0,2 2 huge fan fare and big talking before they leave chaos and pay disputes when they get there allshowandnogo
6,7,0.0,camping tomorrow dannyâ
7,8,0.0,the next school year is the year for exams ð cannot think about that ð school exams hate imagine actorslife revolutionschool girl
8,9,0.0,we won love the land allin cavs champions cleveland clevelandcavaliers â
9,10,0.0,welcome here i m it s so gr8


8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [28]:
def special_symbols_replace(df):
    return df.replace(regex=r'[^a-zA-Z0-9]', value=' ', inplace=True)

special_symbols_replace(combine_df)

combine_df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0.0,thanks for lyft credit i cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0.0,bihday your majesty
3,4,0.0,model i love u take with u all the time in ur
4,5,0.0,factsguide society now motivation
5,6,0.0,2 2 huge fan fare and big talking before they leave chaos and pay disputes when they get there allshowandnogo
6,7,0.0,camping tomorrow danny
7,8,0.0,the next school year is the year for exams cannot think about that school exams hate imagine actorslife revolutionschool girl
8,9,0.0,we won love the land allin cavs champions cleveland clevelandcavaliers
9,10,0.0,welcome here i m it s so gr8


9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [34]:
def numbers_replace(df):
    return df.replace(regex=r'[^a-zA-Z]', value=' ', inplace=True)

numbers_replace(combine_df)

combine_df.head(10)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0.0,thanks for lyft credit i cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0.0,bihday your majesty
3,4,0.0,model i love u take with u all the time in ur
4,5,0.0,factsguide society now motivation
5,6,0.0,huge fan fare and big talking before they leave chaos and pay disputes when they get there allshowandnogo
6,7,0.0,camping tomorrow danny
7,8,0.0,the next school year is the year for exams cannot think about that school exams hate imagine actorslife revolutionschool girl
8,9,0.0,we won love the land allin cavs champions cleveland clevelandcavaliers
9,10,0.0,welcome here i m it s so gr


10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [ ]:
def one_symbol_word_delete(df):
    return df.replace(regex=r'[^a-zA-Z]', value=' ', inplace=True)


def emoticons_replace(df):
    for index, row in df.iterrows():
        text = ''
        for word in row['tweet'].split():
            for emoticon in emoticons:
                if word == emoticon:
                    word = ' '
            text = ' '.join((text, word))
        df.iloc[index, 2] = text
    return df



one_symbol_word_delete(combine_df)

combine_df.head(10)

11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15. Сохраним результат предобработки в pickle-файл.